In [16]:
from bs4 import BeautifulSoup
import pandas as pd

In [17]:
def extract_entries_from_file(file_path):
    entries = []
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
        for entry in soup.find_all('a', attrs={"name": True}):
            try:
                alabama_word = entry['name']
            except:
                continue
            try:
                definition = entry.find_next()
                if (definition.get_text().strip() == "Var. of"):
                    definition = definition.get_text().strip() + " " + definition.find_next().get_text().strip()
                elif (entry.find_next().find_previous(string=True).strip() == "Neg." or entry.find_next().find_previous(string=True).strip() == "Neg.\n ☞" or 'Neg.' in entry.find_next().find_previous(string=True).strip()):
                    definition = "Negative form of " + entry.find_next('a').get_text().strip()
                elif (definition.get_text().strip() == "1" or definition.get_text().strip() == "2"):
                    definition = definition.find_next().get_text().strip()
                elif (definition.get_text().strip()[0] == '(' and file_path == '/Users/jacobfernandes/Desktop/alabamadictionary/DictfilesAlabama/Affixes.txt'):
                    definition = definition.get_text().strip() + " " + definition.find_next().get_text().strip()
                else:
                    definition = definition.get_text().strip()
            except:
                print()
            l_class = ""
            x = None
            if (entry.find_next().find_next().name == "font"):
                text = entry.find_next().find_next()
                if (text.get('color') == '#800080'):
                    l_class = text.get_text().strip()
                    pp = text.find_next()
                    if ',' in pp.get_text().strip():
                        x = (pp.get_text().strip())
                elif (len(definition) >= 3):
                    try:
                        if (definition[0:3] == "to "):
                            l_class = "Other Verb"
                    except:
                        ()
            elif 'Usage' in entry.find_next().find_next().get_text().strip():
                text = entry.find_next().find_next().find_next()
                if (text.get('color') == '#800080'):
                    l_class = text.get_text().strip()
                    pp = text.find_next()
                    if ',' in pp.get_text().strip():
                        x = (pp.get_text().strip())
                elif (len(definition) >= 3):
                    try:
                        if (definition[0:3] == "to "):
                            l_class = "Other Verb"
                    except:
                        ()
            entries.append({
                'Alabama': alabama_word,
                'English': definition,
                'Class' : l_class,
                'Principal Parts': x
            })
    return entries

In [18]:
extract_entries_from_file('/Users/jacobfernandes/Desktop/alabamadictionary/DictfilesAlabama/M.txt')

[{'Alabama': 'ma',
  'English': 'that way; that one (distal; previously mentioned)',
  'Class': '',
  'Principal Parts': None},
 {'Alabama': 'maabàali',
  'English': 'Var. of maabahàali',
  'Class': '',
  'Principal Parts': None},
 {'Alabama': 'maabàalka',
  'English': 'to be lying at a distance (of more than one)',
  'Class': '-LI',
  'Principal Parts': 'maabàalhilka, maabàalhaska'},
 {'Alabama': 'maabahàali',
  'English': 'to push (plural objects) away or toward',
  'Class': '-LI/CHA-',
  'Principal Parts': 'maabahàachi, maabahàahili, maabahàahachi'},
 {'Alabama': 'maabállàaka',
  'English': 'to be lying at a distance (of one)',
  'Class': '-LI',
  'Principal Parts': 'maabállàahiska, -, -'},
 {'Alabama': 'maabatli',
  'English': 'to knock (more than once) at the door',
  'Class': '-LI/CHA-',
  'Principal Parts': 'maabatchi, maabathili, maabathachi'},
 {'Alabama': 'maabinafka',
  'English': 'to erode and collapse',
  'Class': '',
  'Principal Parts': None},
 {'Alabama': 'maachi',
  'E

In [19]:
import os

# Assuming the extract_entries_from_file function is defined as described earlier

folder_path = "DictfilesAlabama"

# Get list of files in the folder
file_names = os.listdir(folder_path)

# Filter only .txt files
txt_files = [file for file in file_names if file.endswith('.txt')]
entries = []
# Iterate over each file and apply the extract_entries_from_file function
for file_name in txt_files:
    file_path = os.path.join(folder_path, file_name)
    entries += extract_entries_from_file(file_path)
    # Do something with the extracted entries, such as saving to a DataFrame
    # For example:
    # df = pd.DataFrame(entries)
    # print(df)


In [20]:
import pandas as pd

In [21]:
entries = pd.DataFrame(entries)
entries

,Alabama,English,Class,Principal Parts
0,kàano,"to be well, all right, feel better",CHA-,None
1,káanon naaɬiíkko,to be unable to talk plainly (as after a strok...,,None
2,káasok,"but, though, however",,None
3,kabòska,caboose,,None
4,kachàali,"to bite repeatedly, chew",-LI/CHA-,"kachàachi, kachàahili, kachàahachi"
...,...,...,...,...
8808,intonotli istiⁿhisli,wheel spokes,,None
8809,intoobi,Var. of intohbi,,None
8810,intòono,Var. of intòhno,,None
8811,intosoyli,to pour for,-LI/AM-/3,"intosoychi, intosoyhili, intosoyhachi"


In [22]:
entries.loc[entries['Alabama'] == '-ba']

,Alabama,English,Class,Principal Parts
6545,-ba,(Imp),,None


In [23]:
entries.to_csv('dictionary.csv')

In [24]:
with open('dictionary.csv', 'r') as f:
    with open('dict.json', 'w') as g:
        g.write("""{ "words" : [""")
        x = pd.read_csv(f)
        for index, row in x.iterrows():
            g.write("""
    { 
        "lemma":\"""" +  str(row['Alabama']) + """\",
        "definition": \"""" + str(row['English']) + """\",
        "class": \"""" + str(row['Class']) + """\",
        "principalPart": \"""" + str(row['Principal Parts']) + """\"
    },""")
        g.write("]}")